# Spatially Enabled Data Frames

#### Setup Environment

In [ ]:
%%html
<style>
.container {width: 90% !important; }
#logo
{
    float: left;
}
#names
{
    float: right;
    
}
#logo2
{
    float: right;
    height=100;
    width=100;
    margin: 5px;
}
#top_menu
{
    margin: 20px auto 0;
    
    height:300px;
    width: 400px;
}
</style>

### Getting Started

## Overview 

### Built on a Pandas
   + Special `spatial` and `geom` namespaces
   + Has a custom `Geometry` column type

### Provides a different geospatial experience:
   + In Memory
   + Fast
   + On the Fly Indexing
   + Multi-platform

### Custom Namespaces!

   + `geom` on Series
   
   ```python
   >>> df['SHAPE'].geom.area
   ```
    
   + `spatial` on the the DataFrame
    
   ```python
   >>> df.spatial.project_as(4326)
   ```

### Other Benefits

- Cross platform spatial analysis
    + Mac, Linux, and Windows

- Multi-geometry engine support

    + Esri Arcpy's Engine
    + Shapely/Geos
    
- Read/write data

    + Fiona, shapefile, and arcpy

## Getting Started

It starts with two imports

In [ ]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

- Loads Pandas
- The `GeoAccessor` and `GeoSeriesAccessor` load the namespaces into Pandas

### Data I/O

#### Consumption

- Feature Layers
- Data Frames
- Feature classes

```python

    from arcgis.features import FeatureLayer
    fl = FeatureLayer(("https://services2.arcgis.com/zPFLSOZ5HzUzzTQb/arcgis"
                       "/rest/services/CensusBlockGroup/FeatureServer/0"))
    sdf1 = pd.DataFrame.spatial.from_layer(fl)
    sdf2 = pd.DataFrame.spatial.from_featureclass("./data/historic_traffic.shp")
    df_earthquakes = pd.read_csv("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.csv")
    sdf3 = pd.DataFrame.spatial.from_xy(df=df_earthquakes, 
                                        x_column="longitude", 
                                        y_column="latitude", 
                                        sr=4326)
```

#### Persistence of Data

- Tables
- Feature Classes
- Services

```python
    item = gis.content.import_data(df=sdf1, title="CensusBlockGroup")
    sdf1.spatial.to_featureclass(location="./data/stage.gdb/census")
```

## Working with the Series Namespace

##### Recall:
   - A series is a 1-D array of values
    

#### Reading a Feature Class

In [ ]:
data = r".\sample_data.gdb/Zoning"
sdf = pd.DataFrame.spatial.from_featureclass(data)
sdf.SHAPE.head()

#### Adding Centroid to Table

In [ ]:
sdf['centers'] = sdf.SHAPE.geom.centroid
sdf['centers'].head()

#### Example Calculating Areas

In [ ]:
sdf.SHAPE.geom.get_area("PLANAR","ACRES").sum()

## Working with the DataFrame Namespace

- The `spatial` name space provides:
    + Dataset level operations
    + Dataset information
    + I/O operations exist here

### `Spatial` Operation Examples

#### Visualize Bounding Box

In [ ]:

sdf = pd.DataFrame.spatial.from_featureclass(data)
sdf.spatial.bbox

#### Full Extent

In [ ]:
sdf.spatial.full_extent

#### Reprojecting

In [ ]:
sdf.spatial.sr

In [ ]:
sdf.spatial.project(3857)
sdf.spatial.sr

## Visualization

- Provides a rich visualization of data
- Data does not have to exist on an Enterprise
- Map both local and service data

### The `plot()` Method

- Mimics syntax and symbology similar to [`matplotlib`](https://matplotlib.org) for visualizing

Some unique characteristics of working with the visualization capabalities on the SDF:
- Uses Pythonic syntax
- Uses the same syntax as visualizing charts on Pandas DataFrames
- Works on features and attributes simultaneously, eliminating to a great extent the need to iterate over all features (rows)
- Handles reading and writing to multiple formats aiding data conversion

### Understanding Renderers

- Renderers define how to visually represent spatial data by defining symbols to represent individual features
- A `SeDF` provides you with functionality to control the way features appear by choosing the symbol the renderer uses

### Supported renderers  

Supports the following renderers:

| Renderer     	| Syntax 	| Explanation                                                                                 	|
|--------------	|--------	|---------------------------------------------------------------------------------------------	|
| Simple       	| 's'    	| renders using one symbol only            	|
| Unique       	| 'u'    	| renders each unique value with a different symbol. Suitable for categorical columns         	|
| Unique       	| 'u-a'    	| renders each unique value with a different symbol using arcade expressions. Suitable for categorical columns         	|
| Class breaks 	| 'c'    	| renders each group of values with a different color or size. Suitable for numerical columns 	|
| Heatmap      	| 'h'    	| renders density of point data as raster of varying colors                                   	|

### Visualization Helper Methods

- Because of the rich visualization possibilities, there are helper methods to assist in the design process

In [ ]:
from arcgis.mapping import show_styles, display_colormaps

In [ ]:
show_styles(geometry_type="POINT")

In [ ]:
%matplotlib inline
display_colormaps()

### Simple Rendering of Data

- Simple renders can be circles, squares, solid colors, etc...
- Single color only

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

In [ ]:
first_map = gis.map('New York, NY')
first_map.basemap = 'dark-gray-vector'
first_map

In [ ]:
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")

In [ ]:
item

In [ ]:
flayer = item.layers[0]

In [ ]:
df = flayer.query(where="AGE_45_54 < 1500", as_df=True)

In [ ]:
df.head()

In [ ]:
df.spatial.plot(map_widget=first_map)

### Advanced Simple Rendering


In [ ]:
m2 = gis.map('New York, NY')
m2.basemap = 'dark-gray-vector'
m2

In [ ]:
import time
for i in range(10):
    m2.remove_layers()
    df.spatial.plot(map_widget=m2,
                symbol_type='simple',
                renderer_type='s',
                symbol_style='s', # s - for square
                colors='Reds_r',
                cstep=i,
                outline_color='YlOrBr',
                marker_size=10)
    time.sleep(2)

### Class Break Renderer

In [ ]:
m3 = gis.map('Reno, NV', zoomlevel=4)
m3.center = [39,-98]
m3

In [ ]:
df.spatial.plot(map_widget = m3,
                renderer_type='u', # specify the unique value renderer using its notation 'u'
                col='ST'  # column to get unique values from
               )

### Class Break Renderer


In [ ]:
m4 = gis.map('Reno, NV', zoomlevel=4)
m4.center = [39,-98]
m4

In [ ]:
df.spatial.plot(map_widget=m4,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=20,  # choose the number of classes
               col='POPULATION',  # numeric column to classify
               cmap='gnuplot2_r',  # color map to pick colors from for each class
               alpha=0.7  # specify opacity
               )

In [ ]:
m4.legend = True

### Rendering Polygon Example


In [ ]:
from arcgis.features import FeatureLayer
fl = FeatureLayer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2")
county_sdf = fl.query("STATE_NAME='Washington'", out_sr=4326, as_df=True)
county_sdf.head()

In [ ]:
m5 = gis.map('Seattle, WA', zoomlevel=6)
m5.basemap = 'dark-gray-vector'
m5

In [ ]:
county_sdf.spatial.plot(map_widget=m5,
                        renderer_type='c',  
                        method='esriClassifyNaturalBreaks',  
                        class_count=7,  
                        col='AGE_18_21', 
                        cmap='RdPu',  
                        alpha=0.8,
                        line_width=.25)

## Spatial Index

- Quickly find spatial locations 
    + generalized locations
- Based on Minimum bounding rectangles

### Spatial Index Example

In [ ]:
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
item

In [ ]:
sdf = item.layers[0].query(as_df=True, out_sr=4326) # Major US Cities

In [ ]:
len(sdf)

In [ ]:
index = sdf.spatial.sindex(stype='quadtree')

In [ ]:
nj_cities = index.intersect((-75.55956796790353, 38.928522146813044, 
                             -73.9024505439044, 41.35763612214295))

In [ ]:
m6 = gis.map("New Jersey")
m6

In [ ]:
sdf.iloc[nj_cities].spatial.plot(m6)

#### Notice the Following

- The bounding box returns locations outside of New Jersey
    + This means the the selction by location is **generalized**
    

### Limiting to NJ

In [ ]:
nj_boundary = gis.content.get("e4bb3aeab5e14422be1727ccbe0c1fb9")
nj_bounds = nj_boundary.layers[0].query(out_sr=4326, as_df=True)
geom = nj_bounds.iloc[0].SHAPE
geom

In [ ]:
extent_cities = sdf.iloc[nj_cities].reset_index()
extent_cities.ST.unique()

In [ ]:
m6.remove_layers()
g_query = extent_cities.SHAPE.geom.disjoint(geom) == False
extent_cities[g_query].spatial.plot(m6)

## Spatial Analysis

- We have data, but how to we gain insights into the data?
- Python API provides a wealth of vector based analytics

### Using Geoprocessing Tools

- The work horse of analysis
- Service provide tools not available in the standard libary

In [ ]:
from arcgis import create_viewshed
sub_sdf = sdf.iloc[[2981, 2982, 2983, 2984]].copy()

In [ ]:
sub_sdf

In [ ]:
vs = create_viewshed(input_layer=sub_sdf, 
                     maximum_distance=20, 
                     max_distance_units="Miles")
vs

In [ ]:
m7 = sub_sdf.spatial.plot(
                    symbol_type='simple',
                    symbol_style='d', # d - for diamonds
                    colors='Reds_r',
                    cstep=20,
                    outline_color='Blues',
                    marker_size=20)
m7.basemap = 'dark-gray-vector' 
m7

In [ ]:
m7.add_layer(vs)

In [ ]:
from arcgis.features import FeatureSet
fs = FeatureSet.from_dict(vs.layer['featureSet'])
fs.sdf.spatial.plot(m7)